# Dependecies

In [11]:
import os 
import numpy as np
import pandas as pd

import lightgbm as lgb

import keras 
from keras.layers import *
from keras.optimizers import *
from keras.models import Model
from google.colab import drive

from sklearn.metrics import f1_score 
from sklearn.model_selection import KFold

ModuleNotFoundError: No module named 'numpy'

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_x = pd.read_csv("/content/drive/MyDrive/CSCI 567- DataDriven Competition/Data/train_values.csv")
train_y = pd.read_csv("/content/drive/MyDrive/CSCI 567- DataDriven Competition/Data/train_labels.csv")
test_x  = pd.read_csv("/content/drive/MyDrive/CSCI 567- DataDriven Competition/Data/test_values.csv")
sub_csv = pd.read_csv("/content/drive/MyDrive/CSCI 567- DataDriven Competition/Data/submission_format.csv")

# Geographic Location ID Embedding w/ Autoencoder

In [ ]:
geo1 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_1_id"], test_x["geo_level_1_id"]])))
geo2 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_2_id"], test_x["geo_level_2_id"]])))
geo3 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_3_id"], test_x["geo_level_3_id"]])))

In [ ]:
geo3.shape

(347469, 11861)

In [ ]:
def NET():
    inp = Input((geo3.shape[1],))
    i1 = Dense(16, name="intermediate")(inp)
    x2 = Dense(geo2.shape[1], activation='sigmoid')(i1)
    x1 = Dense(geo1.shape[1], activation='sigmoid')(i1)

    model = Model(inp, [x2,x1])
    model.compile(loss="binary_crossentropy", optimizer="adam")
    return model

In [ ]:
model = NET()
model.fit(geo3, [geo2, geo1], batch_size=128, epochs=10, verbose=2)
model.save("geo_embed.h5")

Epoch 1/20
2715/2715 - 12s - loss: 0.2049 - dense_loss: 0.0535 - dense_1_loss: 0.1514 - 12s/epoch - 4ms/step
Epoch 2/20
2715/2715 - 9s - loss: 0.0528 - dense_loss: 0.0052 - dense_1_loss: 0.0477 - 9s/epoch - 3ms/step
Epoch 3/20
2715/2715 - 9s - loss: 0.0147 - dense_loss: 0.0042 - dense_1_loss: 0.0105 - 9s/epoch - 3ms/step
Epoch 4/20
2715/2715 - 9s - loss: 0.0067 - dense_loss: 0.0035 - dense_1_loss: 0.0033 - 9s/epoch - 3ms/step
Epoch 5/20
2715/2715 - 9s - loss: 0.0043 - dense_loss: 0.0030 - dense_1_loss: 0.0013 - 9s/epoch - 3ms/step
Epoch 6/20
2715/2715 - 9s - loss: 0.0030 - dense_loss: 0.0024 - dense_1_loss: 6.2948e-04 - 9s/epoch - 3ms/step
Epoch 7/20
2715/2715 - 9s - loss: 0.0020 - dense_loss: 0.0016 - dense_1_loss: 3.2409e-04 - 9s/epoch - 3ms/step
Epoch 8/20
2715/2715 - 9s - loss: 0.0013 - dense_loss: 0.0011 - dense_1_loss: 1.7143e-04 - 9s/epoch - 3ms/step
Epoch 9/20
2715/2715 - 9s - loss: 8.3731e-04 - dense_loss: 7.4551e-04 - dense_1_loss: 9.1801e-05 - 9s/epoch - 3ms/step
Epoch 10/20

In [ ]:
# Load GEO-Embed Model
model = NET()
model.load_weights("geo_embed.h5")

In [ ]:
# "Extract Intermediate Layer" Function
from keras import backend as K

get_int_layer_output = K.function([model.layers[0].input],
                                  [model.layers[1].output])

In [ ]:
# Extract GEO-Embeds for all train data points.
# Then assign with train_data

out = []
for dat in geo3[:260601]:
    layer_output = get_int_layer_output([np.expand_dims(dat, axis=0)])[0]
    out.append(layer_output)

out = np.array(out)
out = np.squeeze(out)

train_data = pd.get_dummies(train_x.copy())
train_data = train_data.drop(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'], axis=1)
train_data = train_data.assign(geo_feat1=out[:,0],
                               geo_feat2=out[:,1],
                               geo_feat3=out[:,2],  
                               geo_feat4=out[:,3],
                               geo_feat5=out[:,4],    
                               geo_feat6=out[:,5],
                               geo_feat7=out[:,6],
                               geo_feat8=out[:,7],
                               geo_feat9=out[:,8],
                               geo_feat10=out[:,9],
                               geo_feat11=out[:,10],
                               geo_feat12=out[:,11],
                               geo_feat13=out[:,12],
                               geo_feat14=out[:,13],
                               geo_feat15=out[:,14],           
                               geo_feat16=out[:,15])

In [ ]:
train_data.head()

,building_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,...,geo_feat7,geo_feat8,geo_feat9,geo_feat10,geo_feat11,geo_feat12,geo_feat13,geo_feat14,geo_feat15,geo_feat16
0,802906,2,30,6,5,1,1,0,0,0,...,2.453606,0.663133,-2.742998,0.942900,1.713542,-0.190193,0.025504,-0.966479,0.529254,2.353528
1,28830,2,10,8,7,0,1,0,0,0,...,0.715540,0.513491,-1.326827,1.054570,0.801540,0.628300,1.917783,-0.596464,-0.156108,2.317471
2,94947,2,10,5,5,0,1,0,0,0,...,2.059289,-2.129754,-0.356203,2.584619,1.233232,0.067972,1.685591,-1.354865,-2.365149,2.443245
3,590882,2,10,6,5,0,1,0,0,0,...,2.399885,-1.546041,-2.667778,2.722708,-1.861792,-0.204627,1.238448,-2.554839,0.727398,0.201416
4,201944,3,30,8,9,1,0,0,0,0,...,2.398724,-2.160223,-3.339521,0.717559,1.173417,1.408875,1.921816,-0.356582,-0.942084,2.903587


In [ ]:
train_data.columns

Index(['building_id', 'count_floors_pre_eq', 'age', 'area_percentage',
       'height_percentage', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'land_surface_condition_n',
       'land_surface_condition_o', 'land_surface_condition

In [ ]:
# Extract GEO-Embeds for all test data points.
# Then assign with test_data

out = []
for dat in geo3[260601:]:
    layer_output = get_int_layer_output([np.expand_dims(dat, axis=0)])[0]
    out.append(layer_output)

out = np.array(out)
out = np.squeeze(out)

test_data = pd.get_dummies(test_x.copy())
test_data = test_data.drop(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'], axis=1)
test_data = test_data.assign(geo_feat1=out[:,0],
                             geo_feat2=out[:,1],
                             geo_feat3=out[:,2],  
                             geo_feat4=out[:,3],
                             geo_feat5=out[:,4],    
                             geo_feat6=out[:,5],
                             geo_feat7=out[:,6],
                             geo_feat8=out[:,7],
                             geo_feat9=out[:,8],
                             geo_feat10=out[:,9],
                             geo_feat11=out[:,10],
                             geo_feat12=out[:,11],
                             geo_feat13=out[:,12],
                             geo_feat14=out[:,13],
                             geo_feat15=out[:,14],           
                             geo_feat16=out[:,15])

In [ ]:
test_data.head()

,building_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,...,geo_feat7,geo_feat8,geo_feat9,geo_feat10,geo_feat11,geo_feat12,geo_feat13,geo_feat14,geo_feat15,geo_feat16
0,300051,3,20,7,6,0,1,0,0,0,...,1.031540,0.803578,-1.455744,0.727538,1.360039,-0.676458,2.022829,-1.670265,-1.052400,0.869290
1,99355,2,25,13,5,0,1,0,0,0,...,0.605904,0.483528,-0.276234,1.613376,0.832527,-0.034497,-0.627443,-1.223164,-1.356576,2.693806
2,890251,2,5,4,5,0,1,0,0,0,...,1.681867,-0.974341,-1.787380,1.650865,0.302481,-0.506048,-0.449898,-0.887599,0.089050,0.341977
3,745817,1,0,19,3,0,0,0,0,0,...,1.749433,-3.039352,-2.066612,2.769187,-3.320488,1.233824,2.609901,-2.370863,-0.513273,1.730830
4,421793,3,15,8,7,0,1,0,0,0,...,1.211849,-0.898945,-2.475519,-0.408154,1.331401,-1.307947,1.678209,-1.892060,-2.293697,1.217695


In [ ]:
test_data.columns

Index(['building_id', 'count_floors_pre_eq', 'age', 'area_percentage',
       'height_percentage', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'land_surface_condition_n',
       'land_surface_condition_o', 'land_surface_condition

In [ ]:
def threshold_arr(array):
    # Get major confidence-scored predicted value.
    new_arr = []
    for ix, val in enumerate(array):
        loc = np.array(val).argmax(axis=0)
        k = list(np.zeros((len(val))))
        k[loc]=1
        new_arr.append(k)
        
    return np.array(new_arr)

# LightGBM Training

In [ ]:
SEED = 1881
save_path = "/content/drive/MyDrive/CSCI 567- DataDriven Competition/Data/models"
y = np.array(train_y["damage_grade"])-1

df = train_data.drop(["building_id"], axis=1)
x = np.array(df)



In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
for ix, (train_index, test_index) in enumerate(kf.split(x)):
    # lgb_params = {
    #     "objective" : "multiclass",
    #     "num_class":3,
    #     "metric" : "multi_error",
    #     "boosting": 'gbdt',
    #     "max_depth" : -1,
    #     "num_leaves" : 30,
    #     "learning_rate" : 0.1,
    #     "feature_fraction" : 0.5,
    #     "min_sum_hessian_in_leaf" : 0.1,
    #     "max_bin":8192,
    #     "verbosity" : 1,
    #     "num_threads":6,
    #     "seed": SEED
    # }
    lgb_params = {'learning_rate':.05,'boosting_type':'gbdt','objective':'multiclass','num_boost_round':1500,'num_leaves':300,'max_depth':10,'max_bin':8192,'num_class':3,
          'feature_fraction':0.5}

    x_train, x_val, y_train, y_val= x[train_index], x[test_index], y[train_index], y[test_index]

    train_data = lgb.Dataset(x_train, label=y_train)
    val_data   = lgb.Dataset(x_val, label=y_val)

    lgb_clf = lgb.train(lgb_params,
                        train_data,
                        20000,
                        valid_sets = [val_data],
                        early_stopping_rounds=3000,
                        verbose_eval = 1000)

    y_pred = lgb_clf.predict(x_val)
    
    y_pred_labels = np.argmax(y_pred, axis=1)
    print("F1-MICRO SCORE: ", f1_score(y_val, y_pred_labels, average='micro'))

    lgb_clf.save_model(os.path.join(save_path, f'model{ix}.txt'))

/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.042473 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4272
[LightGBM] [Info] Number of data points in the train set: 208480, number of used features: 80
[LightGBM] [Info] Start training from score -2.333567
[LightGBM] [Info] Start training from score -0.564567
[LightGBM] [Info] Start training from score -1.095284
[LightGBM] [Info] Start training from score -34.538776
Training until validation scores don't improve for 3000 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057037 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4274
[LightGBM] [Info] Number of data points in the train set: 208481, number of used features: 80
[LightGBM] [Info] Start training from score -2.343166
[LightGBM] [Info] Start training from score -0.562987
[LightGBM] [Info] Start training from score -1.095203
[LightGBM] [Info] Start training from score -34.538776
Training until validation scores don't improve for 3000 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.039374 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4268
[LightGBM] [Info] Number of data points in the train set: 208481, number of used features: 79
[LightGBM] [Info] Start training from score -2.341369
[LightGBM] [Info] Start training from score -0.564496
[LightGBM] [Info] Start training from score -1.093154
[LightGBM] [Info] Start training from score -34.538776
Training until validation scores don't improve for 3000 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017523 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4269
[LightGBM] [Info] Number of data points in the train set: 208481, number of used features: 79
[LightGBM] [Info] Start training from score -2.335205
[LightGBM] [Info] Start training from score -0.564269
[LightGBM] [Info] Start training from score -1.095318
[LightGBM] [Info] Start training from score -34.538776
Training until validation scores don't improve for 3000 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021306 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4274
[LightGBM] [Info] Number of data points in the train set: 208481, number of used features: 80
[LightGBM] [Info] Start training from score -2.342567
[LightGBM] [Info] Start training from score -0.563830
[LightGBM] [Info] Start training from score -1.093942
[LightGBM] [Info] Start training from score -34.538776
Training until validation scores don't improve for 3000 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

# Create Submission File

In [ ]:
# Load all LightGB Models and concatenate.
models = []
for i in range(5):
    model = lgb.Booster(model_file=os.path.join(save_path, f'model{i}.txt'))

    y_pred = model.predict(x)
    
    y_pred_labels = np.argmax(y_pred, axis=1)
    score  = f1_score(y, y_pred_labels, average='micro')
    print("F1-MICRO SCORE: ", score)
    models.append(model)

F1-MICRO SCORE:  0.81931381690784
F1-MICRO SCORE:  0.816009915541383
F1-MICRO SCORE:  0.8154151365497446
F1-MICRO SCORE:  0.8184734517519119
F1-MICRO SCORE:  0.8195939386264827


In [ ]:
def ensemble(models, x):
    # Ensemble K-Fold CV models with adding all confidence score by class.
    y_preds = []
    
    for model in models:
        y_pred = model.predict(x)
        y_preds.append(y_pred)
        
    init_y_pred = y_preds[0]
    for ypred in y_preds[1:]:
        init_y_pred += ypred
        
    y_pred = threshold_arr(init_y_pred)
    
    return y_pred

In [ ]:
df = test_data.drop(["building_id"], axis=1)
x = np.array(df)

In [ ]:
y_pred = ensemble(models, x)
y_pred = y_pred.argmax(axis=1)+1

In [ ]:
sub_csv["damage_grade"] = y_pred
sub_csv.to_csv("submission.csv", index=False)

In [ ]:
from google.colab import files
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>